## 7. Polars Cleaning messy data

In [2]:
!wget https://frenzy86.s3.eu-west-2.amazonaws.com/python/data/311-service-requests.csv

--2024-11-07 07:33:22--  https://frenzy86.s3.eu-west-2.amazonaws.com/python/data/311-service-requests.csv
Resolving frenzy86.s3.eu-west-2.amazonaws.com (frenzy86.s3.eu-west-2.amazonaws.com)... 3.5.244.164, 52.95.142.50, 52.95.149.58, ...
Connecting to frenzy86.s3.eu-west-2.amazonaws.com (frenzy86.s3.eu-west-2.amazonaws.com)|3.5.244.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55052764 (53M) [text/csv]
Saving to: ‘311-service-requests.csv’

311-service-request 100%[===================>]  52.50M  21.2MB/s    in 2.5s    

2024-11-07 07:33:26 (21.2 MB/s) - ‘311-service-requests.csv’ saved [55052764/55052764]



In [3]:
path = "311-service-requests.csv"

#path = "https://frenzy86.s3.eu-west-2.amazonaws.com/python/data/311-service-requests.csv"


In [4]:
import polars as pl
import polars.selectors as cs
import seaborn as sbn
import matplotlib.pyplot as plt

# Make the graphs a bit prettier, and bigger
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 5)
plt.rcParams['font.family'] = 'sans-serif'

print(pl.__version__)

1.9.0


One of the main problems with messy data is: how do you know if it's messy or not?

We're going to use the NYC 311 service request dataset again here, since it's big and a bit unwieldy. If we try to read it in polars, we immediately run into an error. Polars cannot infer the data type of the data in the csv file:

In [5]:
requests = pl.read_csv(path)

ComputeError: could not parse `N/A` as dtype `i64` at column 'Incident Zip' (column number 9)

The current offset in the file is 3842181 bytes.

You might want to try:
- increasing `infer_schema_length` (e.g. `infer_schema_length=10000`),
- specifying correct dtype with the `dtypes` argument
- setting `ignore_errors` to `True`,
- adding `N/A` to the `null_values` list.

Original error: ```remaining bytes non-empty```

We can force polars to try harder to infer the data type by setting `infer_schema_length` to `None`. Looking at the schema below, I can see that `Incident Zip` was parsed as a `string`. That doesn't look right.

In [7]:
requests = pl.read_csv(path, infer_schema_length=None)
display(requests.head())
display(requests.schema)

Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Action Updated Date,Community Board,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Park Facility Name,Park Borough,School Name,School Number,School Region,School Code,School Phone Number,School Address,School City,School State,School Zip,School Not Found,School or Citywide Complaint,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str
26589651,"""10/31/2013 02:08:41 AM""",null,"""NYPD""","""New York City Police Departmen…","""Noise - Street/Sidewalk""","""Loud Talking""","""Street/Sidewalk""","""11432""","""90-03 169 STREET""","""169 STREET""","""90 AVENUE""","""91 AVENUE""",null,null,"""ADDRESS""","""JAMAICA""",null,"""Precinct""","""Assigned""","""10/31/2013 10:08:41 AM""","""10/31/2013 02:35:17 AM""","""12 QUEENS""","""QUEENS""",1042027,197389,"""Unspecified""","""QUEENS""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""N""",null,null,null,null,null,null,null,null,null,null,null,40.708275,-73.791604,"""(40.70827532593202, -73.791603…"
26593698,"""10/31/2013 02:01:04 AM""",null,"""NYPD""","""New York City Police Departmen…","""Illegal Parking""","""Commercial Overnight Parking""","""Street/Sidewalk""","""11378""","""58 AVENUE""","""58 AVENUE""","""58 PLACE""","""59 STREET""",null,null,"""BLOCKFACE""","""MASPETH""",null,"""Precinct""","""Open""","""10/31/2013 10:01:04 AM""",null,"""05 QUEENS""","""QUEENS""",1009349,201984,"""Unspecified""","""QUEENS""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""N""",null,null,null,null,null,null,null,null,null,null,null,40.721041,-73.909453,"""(40.721040535628305, -73.90945…"
26594139,"""10/31/2013 02:00:24 AM""","""10/31/2013 02:40:32 AM""","""NYPD""","""New York City Police Departmen…","""Noise - Commercial""","""Loud Music/Party""","""Club/Bar/Restaurant""","""10032""","""4060 BROADWAY""","""BROADWAY""","""WEST 171 STREET""","""WEST 172 STREET""",null,null,"""ADDRESS""","""NEW YORK""",null,"""Precinct""","""Closed""","""10/31/2013 10:00:24 AM""","""10/31/2013 02:39:42 AM""","""12 MANHATTAN""","""MANHATTAN""",1001088,246531,"""Unspecified""","""MANHATTAN""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""N""",null,null,null,null,null,null,null,null,null,null,null,40.84333,-73.939144,"""(40.84332975466513, -73.939143…"
26595721,"""10/31/2013 01:56:23 AM""","""10/31/2013 02:21:48 AM""","""NYPD""","""New York City Police Departmen…","""Noise - Vehicle""","""Car/Truck Horn""","""Street/Sidewalk""","""10023""","""WEST 72 STREET""","""WEST 72 STREET""","""COLUMBUS AVENUE""","""AMSTERDAM AVENUE""",null,null,"""BLOCKFACE""","""NEW YORK""",null,"""Precinct""","""Closed""","""10/31/2013 09:56:23 AM""","""10/31/2013 02:21:10 AM""","""07 MANHATTAN""","""MANHATTAN""",989730,222727,"""Unspecified""","""MANHATTAN""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""N""",null,null,null,null,null,null,null,null,null,null,null,40.778009,-73.980213,"""(40.7780087446372, -73.9802134…"
26590930,"""

Schema([('Unique Key', Int64),
        ('Created Date', String),
        ('Closed Date', String),
        ('Agency', String),
        ('Agency Name', String),
        ('Complaint Type', String),
        ('Descriptor', String),
        ('Location Type', String),
        ('Incident Zip', String),
        ('Incident Address', String),
        ('Street Name', String),
        ('Cross Street 1', String),
        ('Cross Street 2', String),
        ('Intersection Street 1', String),
        ('Intersection Street 2', String),
        ('Address Type', String),
        ('City', String),
        ('Landmark', String),
        ('Facility Type', String),
        ('Status', String),
        ('Due Date', String),
        ('Resolution Action Updated Date', String),
        ('Community Board', String),
        ('Borough', String),
        ('X Coordinate (State Plane)', Int64),
        ('Y Coordinate (State Plane)', Int64),
        ('Park Facility Name', String),
        ('Park Borough', String),
      

# 7.1 How do we know if it's messy?

We're going to look at a few columns here. I know already that there are some problems with the zip code, so let's look at that first.

To get a sense for whether a column has problems, I usually use `.unique()` to look at all its values. If it's a numeric column, I'll instead plot a histogram to get a sense of the distribution.

When we look at the unique values in "Incident Zip", it quickly becomes clear that this is a mess.

Some of the problems:

* Some have been parsed as strings, and some as floats
* There are `nan`s
* Some of the zip codes are `29616-0759` or `83`
* There are some N/A values that polars didn't recognize, like 'N/A' and 'NO CLUE'

What we can do:

* Normalize 'N/A' and 'NO CLUE' into regular nan values
* Look at what's up with the 83, and decide what to do
* Make everything strings

In [8]:
requests['Incident Zip'].unique().sort()

Incident Zip
str
null
"""00000"""
"""000000"""
"""00083"""
"""02061"""
…
"""90010"""
"""92123"""
"""N/A"""


# 7.2 Fixing the null_values and string/float confusion

We can pass a `null_values` option to `pl.read_csv` to clean this up a little bit. We can also specify that the type of Incident Zip is a string, not a float.

In [10]:
null_values = ['NO CLUE', 'N/A', '0', 'NA']
requests = pl.read_csv(path, null_values=null_values, dtypes={'Incident Zip':pl.String})
requests['Incident Zip'].unique().sort()

<ipython-input-10-1164ddf8533a>:2: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  requests = pl.read_csv(path, null_values=null_values, dtypes={'Incident Zip':pl.String})


Incident Zip
str
null
"""00000"""
"""000000"""
"""00083"""
"""02061"""
…
"""70711"""
"""77056"""
"""77092-2016"""


# 7.3 What's up with the dashes?

In [11]:
rows_with_dashes = requests.filter(
    pl.col('Incident Zip').str.contains('-')
)
print('number of zip codes with dashes: ', rows_with_dashes.height)
rows_with_dashes.head()

number of zip codes with dashes:  5


Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Action Updated Date,Community Board,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Park Facility Name,Park Borough,School Name,School Number,School Region,School Code,School Phone Number,School Address,School City,School State,School Zip,School Not Found,School or Citywide Complaint,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str
26550551,"""10/24/2013 06:16:34 PM""",null,"""DCA""","""Department of Consumer Affairs""","""Consumer Complaint""","""False Advertising""",null,"""77092-2016""","""2700 EAST SELTICE WAY""","""EAST SELTICE WAY""",null,null,null,null,null,"""HOUSTON""",null,null,"""Assigned""","""11/13/2013 11:15:20 AM""","""10/29/2013 11:16:16 AM""","""0 Unspecified""","""Unspecified""",null,null,"""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""N""",null,null,null,null,null,null,null,null,null,null,null,null,null,null
26548831,"""10/24/2013 09:35:10 AM""",null,"""DCA""","""Department of Consumer Affairs""","""Consumer Complaint""","""Harassment""",null,"""55164-0737""","""P.O. BOX 64437""","""64437""",null,null,null,null,null,"""ST. PAUL""",null,null,"""Assigned""","""11/13/2013 02:30:21 PM""","""10/29/2013 02:31:06 PM""","""0 Unspecified""","""Unspecified""",null,null,"""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""N""",null,null,null,null,null,null,null,null,null,null,null,null,null,null
26488417,"""10/15/2013 03:40:33 PM""",null,"""TLC""","""Taxi and Limousine Commission""","""Taxi Complaint""","""Driver Complaint""","""Street""","""11549-3650""","""365 HOFSTRA UNIVERSITY""","""HOFSTRA UNIVERSITY""",null,null,null,null,null,"""HEMSTEAD""",null,null,"""Assigned""","""11/30/2013 01:20:33 PM""","""10/16/2013 01:21:39 PM""","""0 Unspecified""","""Unspecified""",null,null,"""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""N""",null,null,null,"""La Guardia Airport""",null,null,null,null,null,null,null,null,null,null
26468296,"""10/10/2013 12:36:43 PM""","""10/26/2013 01:07:07 AM""","""DCA""","""Department of Consumer Affairs""","""Consumer Complaint""","""Debt Not Owed""",null,"""29616-0759""","""PO BOX 25759""","""BOX 25759""",null,null,null,null,null,"""GREENVILLE""",null,null,"""Closed""","""10/26/2013 09:20:28 AM""","""10/26/2013 01:07:07 AM""","""0 Unspecified""","""Unspecified""",null,null,"""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""N""",null,null,null,null,null,null,null,null,null,null,null,null,null,null
26461137,"""10/09/2013 05:23:46 PM""","""10/25/2013 01:06:41 AM""","""DCA""","""Department of Consumer Affairs""","""Consumer Complaint""","""Harassment""",null,"""35209-3114""","""600 BEACON PKWY""","""BEACON PKWY""",null,null,null,null,null,"""BIRMINGHAM""",null,null,"""Closed""","""10/25/2013 02:43:42 PM""","""10/25/2013 01:06:41 AM""","""0 Unspecified""",

I thought these were missing data and originally deleted them. But then my friend Dave pointed out that 9-digit zip codes are normal. Let's look at all the zip codes with more than 5 digits, make sure they're okay, and then truncate them.

In [14]:
requests.filter(
                pl.col('Incident Zip').str.contains('-')
                )['Incident Zip'].unique()

Incident Zip
str


Those all look okay to truncate to me.

In [15]:
requests = requests.with_columns(
                                pl.col('Incident Zip').str.slice(0, 5)
                                )
requests.filter(
                pl.col('Incident Zip').str.contains('-')
                )['Incident Zip'].unique()

Incident Zip
str


Done.

Earlier I thought 00083 was a broken zip code, but turns out Central Park's zip code 00083! Shows what I know. I'm still concerned about the 00000 zip codes, though: let's look at that.

In [17]:
requests.filter(
                pl.col('Incident Zip') == '00000'
                )

Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Action Updated Date,Community Board,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Park Facility Name,Park Borough,School Name,School Number,School Region,School Code,School Phone Number,School Address,School City,School State,School Zip,School Not Found,School or Citywide Complaint,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str
26529313,"""10/22/2013 02:51:06 PM""",null,"""TLC""","""Taxi and Limousine Commission""","""Taxi Complaint""","""Driver Complaint""",null,"""00000""","""EWR EWR""","""EWR""",null,null,null,null,null,"""NEWARK""",null,null,"""Assigned""","""12/07/2013 09:53:51 AM""","""10/23/2013 09:54:43 AM""","""0 Unspecified""","""Unspecified""",null,null,"""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""N""",null,null,null,"""Other""",null,null,null,null,null,null,null,null,null,null
26507389,"""10/17/2013 05:48:44 PM""",null,"""TLC""","""Taxi and Limousine Commission""","""Taxi Complaint""","""Driver Complaint""","""Street""","""00000""","""1 NEWARK AIRPORT""","""NEWARK AIRPORT""",null,null,null,null,null,"""NEWARK""",null,null,"""Assigned""","""12/02/2013 11:59:46 AM""","""10/18/2013 12:01:08 PM""","""0 Unspecified""","""Unspecified""",null,null,"""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""N""",null,null,null,"""Other""",null,null,null,null,null,null,null,null,null,null


This looks bad to me. Let's set these to nan.

In [19]:
requests = requests.with_columns(
                                pl.when(pl.col('Incident Zip') == '00000').then(None).otherwise(pl.col('Incident Zip')).alias('Incident Zip')
                                )
requests.filter(
                pl.col('Incident Zip') == '00000'
                )

Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Action Updated Date,Community Board,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Park Facility Name,Park Borough,School Name,School Number,School Region,School Code,School Phone Number,School Address,School City,School State,School Zip,School Not Found,School or Citywide Complaint,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str


Great. Let's see where we are now:

In [20]:
unique_zips = requests['Incident Zip'].unique().sort()
unique_zips

Incident Zip
str
null
"""00083"""
"""02061"""
"""06901"""
"""07020"""
…
"""70711"""
"""77056"""
"""77092"""


Amazing! This is much cleaner. There's something a bit weird here, though -- I looked up 77056 on Google maps, and that's in Texas.

Let's take a closer look:

In [22]:
requests.lazy().select(
                    'Incident Zip',
                    'Descriptor',
                    'City'
                    ).filter(
                        pl.col('Incident Zip') == "77056"
                    ).sort('Incident Zip').collect()

Incident Zip,Descriptor,City
str,str,str
"""77056""","""Debt Not Owed""","""HOUSTON"""


Okay, there really are requests coming from Houston! Good to know. Filtering by zip code is probably a bad way to handle this -- we should really be looking at the city instead.

In [23]:
requests['City'].str.to_uppercase().value_counts(sort=True)

City,count
str,u32
"""BROOKLYN""",31662
"""NEW YORK""",22664
"""BRONX""",18438
null,12215
"""STATEN ISLAND""",4766
…,…
"""SYRACUSE""",1
"""NANUET""",1
"""FARMINGDALE""",1


There are 12,215 `null` values in the `City` column. Upon closer look, it seems that many of these rows also have missing `Incident Zip` values as well:

In [25]:
requests.select(
                'Incident Zip',
                'Descriptor',
                'City'
                ).filter(
                    pl.col('City').is_null()
                ).sort('Incident Zip')

Incident Zip,Descriptor,City
str,str,str
null,"""Street Light Out""",null
null,"""Street Light Out""",null
null,"""Medicaid""",null
null,"""Controller""",null
null,"""Property Tax Exemption Applica…",null
…,…,…
null,"""Street Light Out""",null
null,"""Street Light Out""",null
null,"""Property Tax Exemption Applica…",null


# 7.4 Putting it together

Here's what we ended up doing to clean up our zip codes, all together:

In [27]:
null_values = ['NO CLUE', 'N/A', '0', 'NA']
requests = (
            pl.scan_csv(path, null_values=null_values, schema_overrides={'Incident Zip':pl.String})
            .with_columns(pl.col('Incident Zip').str.slice(0, 5))
        )
requests = (
            requests
            .with_columns(pl.when(pl.col('Incident Zip') == '00000').then(None).otherwise(pl.col('Incident Zip')).alias('Incident Zip'))
            .filter(pl.col('Incident Zip').is_not_null())
            .collect()
        )

In [28]:
requests['Incident Zip'].unique().sort()

Incident Zip
str
"""00083"""
"""02061"""
"""06901"""
"""07020"""
"""07087"""
…
"""70711"""
"""77056"""
"""77092"""


<style>
    @font-face {
        font-family: "Computer Modern";
        src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');
    }
    div.cell{
        width:800px;
        margin-left:16% !important;
        margin-right:auto;
    }
    h1 {
        font-family: Helvetica, serif;
    }
    h4{
        margin-top:12px;
        margin-bottom: 3px;
       }
    div.text_cell_render{
        font-family: Computer Modern, "Helvetica Neue", Arial, Helvetica, Geneva, sans-serif;
        line-height: 145%;
        font-size: 130%;
        width:800px;
        margin-left:auto;
        margin-right:auto;
    }
    .CodeMirror{
            font-family: "Source Code Pro", source-code-pro,Consolas, monospace;
    }
    .text_cell_render h5 {
        font-weight: 300;
        font-size: 22pt;
        color: #4057A1;
        font-style: italic;
        margin-bottom: .5em;
        margin-top: 0.5em;
        display: block;
    }
    
    .warning{
        color: rgb( 240, 20, 20 )
        }  